In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import glob
import os
from torch.utils.data import DataLoader , ConcatDataset
from torchvision import datasets, transforms
import cv2
import sys
import math
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import cv2
import glob
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import transforms

In [2]:
import cv2
import glob

cat = []
dog = []

def process_images(folder_path, label_list, batch_size=100):
    batch = []
    for f in glob.iglob(folder_path):
        img = cv2.imread(f)
        if img is not None:
            img = cv2.resize(img, (128,128))
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            batch.append(img)
            if len(batch) == batch_size:
                label_list.extend(batch)
                batch = []
    # Add remaining images in the batch
    if batch:
        label_list.extend(batch)

# Process Cats
process_images("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/PetImages/Training/Cat/*.jpg", cat)

# Process Dogs
process_images("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/practice_imgs/Dog/*.jpg", dog)


True code for large size images

In [3]:
cat=[]
dog=[]
batch_1=[]
batch_2=[]

for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/PetImages/Training/Cat/*.jpg"):
    img=cv2.imread(f)
    if img is not None:
        img=cv2.resize(img,(128,128))
        b,g,r=cv2.split(img)
        img=cv2.merge([r,g,b])
        batch_1.append(img)

        if len(batch_1) == 100:
            cat.extend(batch_1)
            batch_1 = []
if batch_1:
    cat.extend(batch_1)


for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/practice_imgs/Dog/*.jpg"):
    img=cv2.imread(f)
    if img is not None:
        img=cv2.resize(img,(128,128))
        b,g,r=cv2.split(img)
        img=cv2.merge([r,g,b])
        batch_2.append(img)

        if len(batch_2) == 100:
            dog.extend(batch_2)
            batch_2 = []
if batch_2:
    dog.extend(batch_2)
    


In [4]:
cat=np.array(cat,dtype=np.float32)

In [5]:
cat.shape

(6794, 128, 128, 3)

In [6]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
class Dataset(object):

  def __getitem__(self, index):
    raise NotImplementedError

  def __len__(self):
    raise NotImplementedError

  def __add__(self, other):
    return ConcatDataset([self, other])

In [8]:
class Cat_Dog(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        cat = []
        dog = []
        batch_1=[]
        batch_2=[]

        # Load and preprocess images
        for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/PetImages/Training/Cat/*.jpg"):
            img = cv2.imread(f)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                b, g, r = cv2.split(img)
                img = cv2.merge([r, g, b])
                img = img.transpose((2, 0, 1))      # Convert to (C, H, W)            
                batch_1.append(img)

                if len(batch_1) == 100:
                    cat.extend(batch_1)
                    batch_1 = []
        if batch_1:
            cat.extend(batch_1)
              
            

        for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/practice_imgs/Dog/*.jpg"):
            img = cv2.imread(f)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                b, g, r = cv2.split(img)
                img = cv2.merge([r, g, b])
                img = img.transpose((2, 0, 1))  # Convert to (C, H, W)
                batch_2.append(img)

                if len(batch_2) == 100:
                    dog.extend(batch_2)
                    batch_2 = []
        if batch_2:
            dog.extend(batch_2)
            


        # Concatenate and create labels
        cat = np.array(cat, dtype=np.float32)
        dog = np.array(dog, dtype=np.float32)

        cat_label = np.ones(cat.shape[0], dtype=np.float32)
        dog_label = np.zeros(dog.shape[0], dtype=np.float32)

        self.images = np.concatenate((cat, dog), axis=0)
        self.labels = np.concatenate((cat_label, dog_label))

        # Normalize images
        self.images /= 255.0

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = torch.tensor(self.images[index], dtype=torch.float32)
        label = torch.tensor(self.labels[index], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 'label': label}
        return sample

    def normalize(self):
      self.images = self.images/255.0


In [9]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn_model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5))

        self.fc_model = nn.Sequential(
        nn.Linear(in_features=256, out_features=120),
        nn.Tanh(),
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh(),
        nn.Linear(in_features=84, out_features=1))

    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        x = F.sigmoid(x)

        return x

In [10]:
cat_dataset = Cat_Dog()
cat_dataset.normalize()
device = torch.device('cuda:0')
model = CNN().to(device)


In [11]:
names={0:'Cat', 1:'Dog'}
dataloader = DataLoader(cat_dataset, batch_size=32, shuffle=False)

In [12]:
model.eval()
outputs = []
y_true = []
with torch.no_grad():
    for D in dataloader:
        image = D['image'].to(device)
        label = D['label'].to(device)

        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())


In [13]:
outputs = np.concatenate( outputs, axis=0 ).squeeze()
y_true = np.concatenate( y_true, axis=0 ).squeeze()

In [14]:
def threshold(scores,threshold=0.50, minimum=0, maximum = 1.0):
    x = np.array(list(scores))
    x[x >= threshold] = maximum
    x[x < threshold] = minimum
    return x

In [15]:
accuracy_score(y_true, threshold(outputs))

0.9963337732805396

In [16]:
eta = 0.0001
EPOCH = 500
optimizer = torch.optim.Adam(model.parameters(), lr=eta)
dataloader = DataLoader(cat_dataset, batch_size=32, shuffle=True)
model.train()

CNN(
  (cnn_model): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=5, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=5, padding=0)
  )
  (fc_model): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Tanh()
    (4): Linear(in_features=84, out_features=1, bias=True)
  )
)

In [17]:
for epoch in range(1, EPOCH):
    losses = []
    for D in dataloader:
        optimizer.zero_grad()
        data = D['image'].to(device)
        label = D['label'].to(device)
        y_hat = model(data)
        # define loss function
        error = nn.BCELoss()
        loss = torch.sum(error(y_hat.squeeze(), label))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    if (epoch+1) % 10 == 0:
        print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch+1, np.mean(losses)))

Train Epoch: 10	Loss: 0.024239
Train Epoch: 20	Loss: 0.024218
Train Epoch: 30	Loss: 0.024205
Train Epoch: 40	Loss: 0.024202
Train Epoch: 50	Loss: 0.024212
Train Epoch: 60	Loss: 0.024222
Train Epoch: 70	Loss: 0.024167
Train Epoch: 80	Loss: 0.024164
Train Epoch: 90	Loss: 0.024174
Train Epoch: 100	Loss: 0.024164
Train Epoch: 110	Loss: 0.024169
Train Epoch: 120	Loss: 0.024183
Train Epoch: 130	Loss: 0.024183
Train Epoch: 140	Loss: 0.024175
Train Epoch: 150	Loss: 0.024180
Train Epoch: 160	Loss: 0.024181
Train Epoch: 170	Loss: 0.024173
Train Epoch: 180	Loss: 0.024183
Train Epoch: 190	Loss: 0.024168
Train Epoch: 200	Loss: 0.024154
Train Epoch: 210	Loss: 0.024168
Train Epoch: 220	Loss: 0.024173
Train Epoch: 230	Loss: 0.024167
Train Epoch: 240	Loss: 0.024189
Train Epoch: 250	Loss: 0.024166
Train Epoch: 260	Loss: 0.024155
Train Epoch: 270	Loss: 0.024168
Train Epoch: 280	Loss: 0.024171
Train Epoch: 290	Loss: 0.024153
Train Epoch: 300	Loss: 0.024159
Train Epoch: 310	Loss: 0.024164
Train Epoch: 320	

In [20]:
model.eval()

dataloader = DataLoader(cat_dataset, batch_size=32, shuffle=False)
outputs=[]
y_true = []
with torch.no_grad():
    for D in dataloader:
        image =  D['image'].to(device)
        label = D['label'].to(device)

        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())

outputs = np.concatenate( outputs, axis=0 )
y_true = np.concatenate( y_true, axis=0 )

In [21]:
accuracy_score(y_true, threshold(outputs))

0.9963337732805396

In [22]:
class Test_data(Dataset):
    def __init__(self, transform=None):
        self.transform = transform

        cat = []
        dog = []
        batch_1=[]
        batch_2=[]

        # Load and preprocess images
        for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/PetImages/Test/Cat/*.jpg"):
            img = cv2.imread(f)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                b, g, r = cv2.split(img)
                img = cv2.merge([r, g, b])
                img = img.transpose((2, 0, 1))      # Convert to (C, H, W)            
                batch_1.append(img)

                if len(batch_1) == 100:
                    cat.extend(batch_1)
                    batch_1 = []
        if batch_1:
            cat.extend(batch_1)
              
            

        for f in glob.iglob("C:/Users/ACER/Desktop/Final Year Projects/Vs_code/Jupyter notebook/PetImages/Test/Dog/*.jpg"):
            img = cv2.imread(f)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                b, g, r = cv2.split(img)
                img = cv2.merge([r, g, b])
                img = img.transpose((2, 0, 1))  # Convert to (C, H, W)
                batch_2.append(img)

                if len(batch_2) == 100:
                    dog.extend(batch_2)
                    batch_2 = []
        if batch_2:
            dog.extend(batch_2)
            


        # Concatenate and create labels
        cat = np.array(cat, dtype=np.float32)
        dog = np.array(dog, dtype=np.float32)

        cat_label = np.ones(cat.shape[0], dtype=np.float32)
        dog_label = np.zeros(dog.shape[0], dtype=np.float32)

        self.images = np.concatenate((cat, dog), axis=0)
        self.labels = np.concatenate((cat_label, dog_label))

        # Normalize images
        self.images /= 255.0

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = torch.tensor(self.images[index], dtype=torch.float32)
        label = torch.tensor(self.labels[index], dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        sample = {'image': image, 'label': label}
        return sample

    def normalize(self):
      self.images = self.images/255.0


In [23]:
test_cat=Test_data()

In [24]:
model.eval()

dataloader = DataLoader(test_cat, batch_size=32, shuffle=False)
outputs=[]
y_true = []
with torch.no_grad():
    for D in dataloader:
        image =  D['image'].to(device)
        label = D['label'].to(device)

        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())

outputs = np.concatenate( outputs, axis=0 )
y_true = np.concatenate( y_true, axis=0 )

In [25]:
accuracy_score(y_true, threshold(outputs))


0.6915322580645161